In [10]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import xgboost, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pickle



In [11]:
trainDF =pd.read_csv('data_ful_cleard.csv',encoding='utf-8',delimiter=';',error_bad_lines=False)

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [12]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [13]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

### Word Embeddings### 

In [14]:
# load the pre-trained word-embedding vectors 


embeddings_index = {}
for i, line in enumerate(open('wiki.ar.vec',encoding="utf-8")):
    try:
        values = line.split()
        embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
    except ValueError :
        continue



# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        


### Topic Models as features

In [19]:
def train_model(name,classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False, ep = 1):
    if is_neural_net:
        # fit the training dataset on the classifier
        classifier.fit(feature_vector_train, label, epochs = ep)
        classifier.save("models/"+name+".h5")
    else :
        # fit the training dataset on the classifier
        classifier.fit(feature_vector_train, label)
        pickle.dump(classifier,open("models/"+name+".pkl","wb"))

    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)

    return metrics.accuracy_score(predictions, valid_y)

### Naive Bayes

In [7]:
# Naive Bayes on Count Vectors
accuracy = train_model("NB1_CountVectors",naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model("NB2_WordLevelTFIDF",naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model("NB3_NGramVectors",naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model("NB4_CharLevelVectors",naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.7289566970181414
NB, WordLevel TF-IDF:  0.5642941544449851
NB, N-Gram Vectors:  0.3762076874956558
NB, CharLevel Vectors:  0.6201084312226316


### Linear Classifier

In [8]:
# Linear Classifier on Count Vectors
accuracy = train_model("LR1_CountVectors",linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model("LR2_WordLevelTFIDF",linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model("LR3_NGramVectors",linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model("LR4_CharLevelVectors",linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.7041078751650796
LR, WordLevel TF-IDF:  0.5774310141099603
LR, N-Gram Vectors:  0.3758601515256829
LR, CharLevel Vectors:  0.6691457565858067


### Implementing a SVM Model

In [ ]:
import datetime
# SVM on Ngram Level TF IDF Vectors
t = datetime.datetime.now()
accuracy = train_model("SVM1_CountVectors",svm.SVC(), xtrain_count, train_y, xvalid_count)
print ("SVM, Count Vectors: ", accuracy)
t1 =datetime.datetime.now()
accuracy = train_model("SVM2_WordLevelTFIDF",svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)
t2 = datetime.datetime.now()
accuracy = train_model("SVM3_NGramVectors",svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)
t3 =datetime.datetime.now()
accuracy = train_model("SVM4_CharLevelVectors",svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM, CharLevel Vectors: ", accuracy)
t4 =datetime.datetime.now()

print("SVM, Count Vectors time:"+str(t1 - t))
print("SVM, WordLevel TF-IDF:"+str(t1 - t))
print("SVM, N-Gram Vectors :"+str(t1 - t))
print("SVM, CharLevel Vectors:"+str(t1 - t))

### Bagging Model

In [ ]:
# RF on Count Vectors
accuracy = train_model("RF1_CountVectors",ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print ("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model("RF2_WordLevelTFIDF",ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

### Boosting Model

In [16]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model("Xgb1_CountVectors",xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print ("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model("Xgb2_WordLevelTFIDF",xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model("Xgb3_CharLevelVectors",xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

Xgb, Count Vectors:  0.4493640091749496
Xgb, WordLevel TF-IDF:  0.44786960450406615
Xgb, CharLevel Vectors:  0.5706193090984917


### Shallow Neural Networks

In [20]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model("NN_NgramLevelTFIDFVectors",classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True, ep=5)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

Epoch 1/5
86320/86320 [==============================] - 33s 382us/step - loss: -24.9314
Epoch 2/5
86320/86320 [==============================] - 33s 377us/step - loss: -27.6077
Epoch 3/5
86320/86320 [==============================] - 28s 329us/step - loss: -27.7438
Epoch 4/5
86320/86320 [==============================] - 31s 357us/step - loss: -27.8060
Epoch 5/5
86320/86320 [==============================] - 30s 343us/step - loss: -27.8480
NN, Ngram Level TF IDF Vectors 0.19197886981302564


### Convolutional Neural Network

In [17]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model("CNN_WordEmbeddings",classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True,ep=5)
print ("CNN, Word Embeddings",  accuracy)

Epoch 1/1
86320/86320 [==============================] - 218s 3ms/step - loss: -26.6635
CNN, Word Embeddings 0.1870438590394106


### Recurrent Neural Network – LSTM

In [18]:
def create_rnn_lstm():
        # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy = train_model("RNN_LSTM_WordEmbeddings",classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True,ep=5)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/1
86320/86320 [==============================] - 748s 9ms/step - loss: -26.7312
RNN-LSTM, Word Embeddings 0.1870438590394106


### Recurrent Neural Network – GRU

In [18]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_gru()
accuracy = train_model("RNN_GRU_WordEmbeddings",classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True,ep=5)
print ("RNN-GRU, Word Embeddings",  accuracy)

NameError: name 'word_index' is not defined

### Bidirectional RNN

In [19]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()
accuracy = train_model("RNN_Bidirectional_WordEmbeddings",classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True,ep=5)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

NameError: name 'word_index' is not defined

### Recurrent Convolutional Neural Network

In [20]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model("CNN_WordEmbeddings",classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True,ep=5)
print ("CNN, Word Embeddings",  accuracy)

NameError: name 'word_index' is not defined